In [2]:
import gc
import os
import random

import lightgbm as lgb
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

seed = 0
random.seed(seed)

In [3]:
df_train = pd.read_csv("data/train.csv")

building = pd.read_csv('data/building_metadata_external.csv')
le = LabelEncoder()
building.primary_use = le.fit_transform(building.primary_use)

weather_train = pd.read_csv('data/weather_train.csv')
weather_test = pd.read_csv('data/weather_test.csv')

In [4]:
from tools import reduce_mem_usage

df_train = reduce_mem_usage(df_train, use_float16=True)
building = reduce_mem_usage(building, use_float16=True)
weather_train = reduce_mem_usage(weather_train, use_float16=True)
weather_test = reduce_mem_usage(weather_test, use_float16=True)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.09 MB
Memory usage after optimization is: 0.02 MB
Decreased by 76.4%
Memory usage of dataframe is 8.58 MB
Memory usage after optimization is: 2.26 MB
Decreased by 73.7%
Memory usage of dataframe is 17.11 MB
Memory usage after optimization is: 4.50 MB
Decreased by 73.7%


In [5]:
import holidays

in_us = [0,2,3,4,6,8,9,10,13,14,15]
in_ca = [7,11]
in_uk = [1,5]
in_ie = [12]

us_cal =  holidays.US()
ca_cal = holidays.CA()
ie_cal = holidays.IE()
uk_cal = holidays.UK()

In [6]:
def holiday_name(timestamp, site_id):
    if site_id in in_ca:
        return ca_cal.get(timestamp)
    elif site_id in in_uk:
        return uk_cal.get(timestamp)
    elif site_id in in_ie:
        return ie_cal.get(timestamp)
    else:
        return us_cal.get(timestamp)

In [7]:
def transform_holidays(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['holiday_name'] = df.apply(lambda x: holiday_name(x.timestamp, x.site_id), axis=1)
    df['holiday_name'] = df['holiday_name'].astype('category')
    df['holiday_name'] = df['holiday_name'].cat.add_categories(['NONE'])
    df['holiday_name'] = df['holiday_name'].fillna('NONE')
    return df

In [8]:
import holidays

in_us = [0,2,3,4,6,8,9,10,13,14,15]
in_ca = [7,11]
in_uk = [1,5]
in_ie = [12]

us_cal =  holidays.US()
ca_cal = holidays.CA()
ie_cal = holidays.IE()
uk_cal = holidays.UK()

def holiday_name(timestamp, site_id):
    if site_id in in_ca:
        return ca_cal.get(timestamp)
    elif site_id in in_uk:
        return uk_cal.get(timestamp)
    elif site_id in in_ie:
        return ie_cal.get(timestamp)
    else:
        return us_cal.get(timestamp)

def transform_holidays(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['holiday_name'] = df.apply(lambda x: holiday_name(x.timestamp, x.site_id), axis=1)
    df['holiday_name'] = df['holiday_name'].astype('category')
    df['holiday_name'] = df['holiday_name'].cat.add_categories(['NONE'])
    df['holiday_name'] = df['holiday_name'].fillna('NONE')
    return df

In [9]:
weather_train = transform_holidays(weather_train)

weather_test = transform_holidays(weather_test)


In [10]:
class DataPrep:
    def __init__(self):
        pass
    
    def prepare_data(self, X, building_data, weather_data, test=False):

        X.timestamp = pd.to_datetime(X.timestamp)
        X.timestamp = X.timestamp.astype('datetime64[ns]')

        X = X.merge(building_data, on="building_id", how="left")
        X = X.merge(weather_data, on=["site_id", "timestamp"], how="left")

        if not test:
            df_group = X.groupby('building_id')['meter_reading']
            self.building_mean = df_group.mean().astype(np.float16)
            self.building_median = df_group.median().astype(np.float16)
            self.building_min = df_group.min().astype(np.float16)
            self.building_max = df_group.max().astype(np.float16)
            self.building_std = df_group.std().astype(np.float16)

        X['building_mean'] = X['building_id'].map(self.building_mean)
        X['building_median'] = X['building_id'].map(self.building_median)
        X['building_min'] = X['building_id'].map(self.building_min)
        X['building_max'] = X['building_id'].map(self.building_max)
        X['building_std'] = X['building_id'].map(self.building_std)

        X.sort_values("timestamp")
        X.reset_index(drop=True)

        gc.collect()



        X.timestamp = pd.to_datetime(X.timestamp, format="%Y-%m-%d %H:%M:%S")
        X.square_feet = np.log1p(X.square_feet)

        X["hour"] = X.timestamp.dt.hour
        X["weekday"] = X.timestamp.dt.weekday
    #     X['month'] = X.timestamp.dt.month
    #     X['day'] = X.timestamp.dt.day
    #     X = encode_date(X)

        X['is_holiday'] = X['holiday_name'].apply(lambda x: 1 if x != "NONE" else 0)



        if not test:
            X.drop(index=X[(X.meter_reading <=0) &
                           (X.meter == 0)].index, inplace=True)

    #     drop_features = ["timestamp", "wind_direction", "wind_speed", 'holiday_name']
        drop_features = ["wind_direction", "wind_speed", 'holiday_name']



        X.drop(drop_features, axis=1, inplace=True)

        if test:
            row_ids = X.row_id
            X.drop("row_id", axis=1, inplace=True)
            return X, row_ids
        else:
            y = np.log1p(X.meter_reading)
            X.drop(["meter_reading",'timestamp'], axis=1, inplace=True)
            return X, y

In [11]:
prepr = DataPrep()

In [12]:
X_train, y_train = prepr.prepare_data(df_train, building, weather_train, False)
# del df_train, weather_train
gc.collect()

30

In [15]:
df_test = pd.read_csv('data/test.csv')
leak = pd.read_pickle('data/site0.pkl')

df_test = reduce_mem_usage(df_test)

# X_test, row_ids = prepr.prepare_data(df_test, building, weather_test, test=True)

Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.65 MB
Decreased by 71.8%


In [16]:
def apply_leaks(leak, df_test):
    df_test.timestamp = pd.to_datetime(df_test.timestamp)
    df_test.timestamp = df_test.timestamp.astype('datetime64[ns]')
    
    X = df_test.merge(leak, on=["building_id",'timestamp','meter'], how="left")
    X.drop('meter_reading_original', axis=1, inplace=True)
    
    return X

In [17]:
pseudo_df = apply_leaks(leak, df_test)

In [18]:
not_null = pseudo_df[pseudo_df['meter_reading_scraped'].notnull()]

In [19]:
not_null.drop('row_id',axis=1, inplace=True)

In [20]:
not_null['meter_reading'] = not_null['meter_reading_scraped']
not_null.drop('meter_reading_scraped',axis=1, inplace=True)

In [21]:
X_train_pseudo, y_train_pseudo = prepr.prepare_data(not_null, building, weather_test, False)



In [27]:
X_train

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,eui,leed,air_temperature,...,cloud_coverage,precip_depth_1_hr,building_mean,building_median,building_min,building_max,building_std,hour,weekday,is_holiday
45,46,0,0,11,9.110078,2016.0,NaN,167.000000,NaN,25.000000,...,6,0,12.382812,9.718750,0.000000,283.000000,12.867188,0,4,1
72,74,0,0,8,12.867830,1997.0,NaN,2.880859,NaN,25.000000,...,6,0,79.062500,86.000000,0.000000,268.000000,70.437500,0,4,1
91,93,0,0,6,10.415443,1982.0,NaN,118.875000,NaN,25.000000,...,6,0,105.875000,145.750000,0.000000,396.000000,85.562500,0,4,1
103,105,0,1,0,10.832181,NaN,5.0,NaN,NaN,3.800781,...,0,0,78.125000,75.125000,23.296875,174.000000,27.531250,0,4,1
104,106,0,1,0,8.589514,NaN,4.0,NaN,NaN,3.800781,...,0,0,3.292969,0.749023,0.000000,40.000000,5.488281,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216094,1443,0,15,0,10.604404,1913.0,NaN,NaN,NaN,1.700195,...,0,0,89.437500,89.625000,37.437500,136.125000,18.078125,23,5,0
20216095,1444,0,15,1,9.884305,1914.0,NaN,NaN,NaN,1.700195,...,0,0,7.480469,6.675781,0.000000,35.812500,4.386719,23,5,0
20216096,1445,0,15,0,8.366138,NaN,NaN,NaN,NaN,1.700195,...,0,0,4.734375,4.824219,2.400391,10.851562,1.317383,23,5,0
20216098,1447,0,15,4,10.301458,2001.0,NaN,NaN,NaN,1.700195,...,0,0,187.125000,186.625000,132.625000,259.000000,24.203125,23,5,0


In [28]:
X_full = pd.concat([X_train, X_train_pseudo], axis=0)

In [31]:
y_full = pd.concat([y_train, y_train_pseudo])

In [32]:
X_full.shape[0] == y_full.shape[0]

True

In [33]:
pd.to_pickle(X_full, 'pseudo_data/X_pseudo.pkl')
pd.to_pickle(y_full, 'pseudo_data/y_pseudo.pkl')